In [1]:
import numpy as np
import pandas as pd

import time

import tensorflow_decision_forests as tfdf

2023-03-30 10:13:57.633414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load training data

train_starting_time = time.time()
train_df = pd.read_csv('../Data/jet_images_c16_pt0_jedinet_sort_hpT_pcNone_train.csv', nrows = 10000)
train_load_time = time.time()-train_starting_time
print(f'Loaded training sample of {len(train_df)} jets in {train_load_time:.3f}s')

# load testing data

test_starting_time = time.time()
test_df = pd.read_csv('../Data/jet_images_c16_pt0_jedinet_sort_hpT_pcNone_test.csv', nrows = 2500)
test_load_time = time.time()-test_starting_time
print(f'Loaded testing sample of {len(test_df)} jets in {test_load_time:.3f}s')

Loaded training sample of 10000 jets in 31.946s
Loaded testing sample of 2500 jets in 8.298s


In [3]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df_renamed = df.rename(columns={"A": "a", "B": "c"})
print(df)
print('\n-------\n')
print(df_renamed)

   A  B
0  1  4
1  2  5
2  3  6

-------

   a  c
0  1  4
1  2  5
2  3  6


In [15]:
for i in range(int((train_df.shape[1] - 1)/16)):
    train_df = train_df.rename(columns={f"c{i}_cos(theta)":f"c{i}_cos_theta", f"c{i}_cos(theta_rel)":f"c{i}_cos_theta_rel"})

for i in range(int((test_df.shape[1] - 1)/16)):
    test_df = test_df.rename(columns={f"c{i}_cos(theta)":f"c{i}_cos_theta", f"c{i}_cos(theta_rel)":f"c{i}_cos_theta_rel"})

In [16]:
train_df['c0_cos_theta']

0      -0.758426
1      -0.367042
2      -0.175659
3       0.409491
4       0.387506
          ...   
9995   -0.182918
9996   -0.061518
9997   -0.218900
9998    0.870466
9999    0.698825
Name: c0_cos_theta, Length: 10000, dtype: float64

In [17]:
# convert training and testing dataframes to tensorflow datasets
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label='class')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label='class')

In [18]:
BDT_hyperparameters={'num_trees':5,
             'shrinkage':0.2,
             'subsample':0.3,
             'use_hessian_gain':True,
             'growing_strategy':'BEST_FIRST_GLOBAL',
             'max_depth':-1,
             'max_num_nodes':32
            }

In [19]:
# define the model
model = tfdf.keras.GradientBoostedTreesModel(**BDT_hyperparameters)

Use /var/folders/w6/ywy355d53p1fqw8pn8ng6xn80000gn/T/tmp2w8z6862 as temporary training directory


In [20]:
#train the model
starting_time = time.time()
model.fit(train_ds, verbose = 1)
training_time = time.time() - starting_time

Reading training dataset...
Training dataset read in 0:00:10.960216. Found 10000 examples.
Training model...


2023-03-30 10:20:56.073453: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1790] "goss_alpha" set but "sampling_method" not equal to "GOSS".
2023-03-30 10:20:56.073487: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1800] "goss_beta" set but "sampling_method" not equal to "GOSS".
2023-03-30 10:20:56.073496: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1814] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-03-30 10:20:56.090678: I external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:452] Default loss set to MULTINOMIAL_LOG_LIKELIHOOD
2023-03-30 10:20:56.090736: W external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:484] The model configuration specifies 5 trees but computation of the validation loss will o

Model trained in 0:00:07.298975
Compiling model...


2023-03-30 10:21:03.249363: I external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1518] 	num-trees:5 train-loss:1.098411 train-accuracy:0.633765 valid-loss:1.184864 valid-accuracy:0.563207
2023-03-30 10:21:03.252074: I external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:1568] Create final snapshot of the model at iteration 5
2023-03-30 10:21:03.254634: I external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:242] Insufficient number of trees to apply early stopping. Using last loss for metrics.
2023-03-30 10:21:03.254666: I external/ydf/yggdrasil_decision_forests/learner/gradient_boosted_trees/gradient_boosted_trees.cc:309] Final model num-trees:0 valid-loss:1.184864 valid-accuracy:0.563207
[INFO 2023-03-30T10:21:03.295233+02:00 kernel.cc:1214] Loading model from path /var/folders/w6/ywy355d53p1fqw8pn8ng6xn80000gn/T/tmp2w8z6862/model/ with prefix e086b289b

Model compiled.


In [22]:
model_output_dir = '~/Desktop/Models/'
model_name = 'BDT_name_test'

model.save(model_output_dir + model_name)

INFO:tensorflow:Assets written to: ~/Desktop/Models/BDT_name_test/assets


INFO:tensorflow:Assets written to: ~/Desktop/Models/BDT_name_test/assets


In [24]:
model.save("Models/BDT_name_test")

INFO:tensorflow:Assets written to: Models/BDT_name_test/assets


INFO:tensorflow:Assets written to: Models/BDT_name_test/assets
